# Сентимент-анализ отзывов (простая версия)

В этом соревновании вам предстоит прогнозировать по тексту отзыва его тональность: 1 - позитивная, 0 - негативная. В отличие от усложненной версии задачи, здесь вам не требуется самостоятельно собирать обучающую выборку - она есть в предоставляемых вам данных.

![title](imhpng.png)

In [1]:
import pandas as pd 
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [ ]:
predict = pd.read_csv('products_sentiment_sample_submission.csv', index_col='Id')

In [32]:
test = pd.read_csv('products_sentiment_test.tsv', sep = '\t', index_col='Id')

In [33]:
test.head()

text
Id                                                   
0   so , why the small digital elph , rather than ...
1   3/4 way through the first disk we played on it...
2   better for the zen micro is outlook compatibil...
3     6 . play gameboy color games on it with goboy .
4   likewise , i 've heard norton 2004 professiona...

In [30]:
train = pd.read_csv('products_sentiment_train.tsv', sep = '\t', header= None)

In [31]:
train.head()

0  1
0          2 . take around 10,000 640x480 pictures .  1
1  i downloaded a trial version of computer assoc...  1
2  the wrt54g plus the hga7t is a perfect solutio...  1
3  i dont especially like how music files are uns...  0
4  i was using the cheapie pail ... and it worked...  1

In [39]:
train[0] = train[0].str.lower() # Нижний регистр
train[0] = train[0].replace('[^a-zA-Z0-9]', ' ', regex = True) # Замените все, кроме букв и цифр, на пробелы

In [40]:
train.head()

0  1
0          2   take around 10 000 640x480 pictures    1
1  i downloaded a trial version of computer assoc...  1
2  the wrt54g plus the hga7t is a perfect solutio...  1
3  i dont especially like how music files are uns...  0
4  i was using the cheapie pail     and it worked...  1

### Попытка № 1. Базовые параметры

In [45]:
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

In [46]:
for clf in [LogisticRegression]:
    print(cross_val_score(text_classifier(CountVectorizer(), TfidfTransformer(), clf(solver = 'liblinear')), train[0], train[1], cv = 5).mean())

0.7655000000000001


In [48]:
from sklearn.metrics import accuracy_score

In [70]:
# Ответы на тестовой выборке
pipl = text_classifier(CountVectorizer(), TfidfTransformer(), LogisticRegression(solver = 'liblinear'))

predict['y'] = pipl.fit(train[0], train[1]).predict(test['text'])
predict.to_csv('prediction.csv')

## Оценка  - 0.78

### Попытка № 2

In [71]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [97]:
list = []
for clf in [LogisticRegression(solver='liblinear'), LinearSVC(), SGDClassifier()]:
    Score = cross_val_score(
        text_classifier(
            TfidfVectorizer(ngram_range=(3,5), analyzer='char_wb'), clf
        ), 
        train[0], train[1], cv = 5
    ).mean()
    list.append(Score)
print(list)

[0.776, 0.7755, 0.76]


In [98]:
list = []
for clf in [LogisticRegression(solver='liblinear'), LinearSVC(), SGDClassifier()]:
    Score = cross_val_score(
        text_classifier(
            TfidfVectorizer(ngram_range=(1,2)), clf
        ), 
        train[0], train[1], cv = 5
    ).mean()
    list.append(Score)
print(list)

[0.7425, 0.781, 0.7809999999999999]


In [101]:
list = []
for clf in [LogisticRegression(solver='liblinear'), LinearSVC(), SGDClassifier()]:
    Score = cross_val_score(
        text_classifier(
            TfidfVectorizer(ngram_range=(1,3)), clf
        ), 
        train[0], train[1], cv = 5
    ).mean()
    list.append(Score)
print(list)

[0.721, 0.79, 0.7849999999999999]


Исходя из оценок выбираю LinearSVC с ngram_range = (1,3). 

При увеличении min_df с 1 качество снижается. Остальные  параметры не меняю.

In [145]:
# Ответы на тестовой выборке
pipl = text_classifier(TfidfVectorizer(ngram_range=(1,3)), LinearSVC())

predict['y'] = pipl.fit(train[0], train[1]).predict(test['text'])
predict.to_csv('prediction.csv')

## Оценка  - 0.8155